# Importing necessary libraries

In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import precision_recall_fscore_support

In [2]:
df = pd.read_csv('D:/SASTRA UNIVERSITY/Mini_Project/testing.csv')
print(df.shape)
df.head()

(20001, 2)


,urls,type
0,mp3raid.com/music/krizz_kaliko.html,0
1,bopsecrets.org/rexroth/cr/1.htm,0
2,http://www.garage-pirenne.be/index.php?option=...,1
3,http://adventure-nicaragua.net/index.php?optio...,1
4,http://buzzfil.net/m/show-art/ils-etaient-loin...,0


In [3]:
df.type.value_counts()

0    14682
1     5319
Name: type, dtype: int64

# IP Address

In [4]:
import re
def having_ip_address(urls):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', urls)  # Ipv6
    if match:
        return 1
    else:
        return 0
df['ip_in_URL'] = df['urls'].apply(lambda i: having_ip_address(i))

# Random words

In [5]:
def count_random_words(urls):
    li1 = re.findall("[A-Za-z]+",urls)
    return len(li1)
df['random_words'] = df['urls'].apply(lambda i: count_random_words(i))

def avg_len_random(urls):
    sum = 0
    li2 = re.findall("[A-Za-z]+",urls)
    for u in li2:
        sum = sum + len(u)
    return (sum/len(li2))

df['avg_len_random_words'] = df['urls'].apply(lambda i: avg_len_random(i))

# English Words

In [6]:
from enchant import Dict
import swifter
d = Dict("en_US")
pattern = re.compile("[A-Za-z]+") 

def count_english_words(urls): 
    li1 = pattern.findall(urls) 
    return sum([1 for word in li1 if d.check(word)]) 

def avg_len_english(urls): 
    li2 = pattern.findall(urls) 
    english_words = [u for u in li2 if d.check(u)] 
    return sum(map(len, english_words)) / len(english_words) if len(english_words)!=0 else 0

df['english_words'] = df['urls'].swifter.apply(count_english_words)
df['avg_len_english_words'] = df['urls'].swifter.apply(avg_len_english)

Pandas Apply:   0%|          | 0/20001 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20001 [00:00<?, ?it/s]

# Lexical features (;, &, http, https, _, #, @, url_length)

In [7]:
df['http_in_URL'] = df['urls'].apply(lambda i: i.count('http'))
df['semicolon_in_URL'] = df['urls'].apply(lambda i: i.count(';'))
df['AND_in_URL'] = df['urls'].apply(lambda i: i.count('&'))
df['URL_length'] = df['urls'].apply(lambda i: len(str(i)))

df['underscore'] = df['urls'].apply(lambda i: i.count('_'))
df['//'] = df['urls'].apply(lambda i: i.count('//'))
df['@'] = df['urls'].apply(lambda i: i.count('@'))
df['hash'] = df['urls'].apply(lambda i: i.count('#'))
df['https'] = df['urls'].apply(lambda i: i.count('https'))



# Letters 

In [8]:
def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters
df['alphabets_in_URL']= df['urls'].apply(lambda i: letter_count(i))

def uppercase_count(url):
    letters1 = 0
    for i in url:
        if i>='A' and i<='Z':
            letters1 = letters1 + 1
    return letters1
df['uppercase_letters_inURL']= df['urls'].apply(lambda i: uppercase_count(i))

def lowercase_count(url):
    letters2 = 0
    for i in url:
        if i>='a' and i<='z':
            letters2 = letters2 + 1
    return letters2
df['lowercase_letters_inURL']= df['urls'].apply(lambda i: lowercase_count(i))

def uppercase_ratio(url):
    url_len = len(str(url))
    letters1 = 0
    for i in url:
        if i>='A' and i<='Z':
            letters1 = letters1 + 1
    return (letters1/url_len)
df['uppercase_letters_ratio_inURL']= df['urls'].apply(lambda i: uppercase_ratio(i))

def lowercase_ratio(url):
    url_len = len(str(url))
    letters2 = 0
    for i in url:
        if i>='a' and i<='z':
            letters2 = letters2 + 1
    return (letters2/url_len)
df['lowercase_letters_ratio_inURL']= df['urls'].apply(lambda i: lowercase_ratio(i))

# special characters

In [9]:
def spl_char_count(url):
    url_len = len(str(url))
    chars = 0
    for i in url:
        if i=='!' or i=='@' or i=='#' or i=='%' or i=='^' or i=='&' or i=='*' or i=='(' or i==')':
            chars = chars + 1
    return chars
df['Special_char_in_URL']= df['urls'].apply(lambda i: spl_char_count(i))

def spl_char_ratio(url):
    url_len = len(str(url))
    chars = 0
    for i in url:
        if i=='!' or i=='@' or i=='#' or i=='%' or i=='^' or i=='&' or i=='*' or i=='(' or i==')':
            chars = chars + 1
    return chars/url_len
df['Special_char_ratio']= df['urls'].apply(lambda i: spl_char_ratio(i))

# Numbers

In [10]:
def number_ratio(url):
    url_len = len(str(url))
    nums = 0
    for i in url:
        if i>='0' and i<='9':
            nums = nums + 1
    return (nums/url_len)
df['numbers_ratio_inURL']= df['urls'].apply(lambda i: number_ratio(i))

def number_URL(url):
    nums = 0
    for i in url:
        if i>='0' and i<='9':
            nums = nums + 1
    return (nums)
df['numbers_inURL']= df['urls'].apply(lambda i: number_URL(i))

# Alphabets

In [11]:
def alphabet_ratio(url):
    url_len = len(str(url))
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return (letters/url_len)
df['alphabet_ratio_inURL']= df['urls'].apply(lambda i: alphabet_ratio(i))

In [12]:
from urllib.parse import urlparse
from math import log
from string import ascii_lowercase

from tld import get_tld

def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

df['fd_length'] = df['urls'].apply(lambda i: fd_length(i))
df['fd_length2'] = df['urls'].apply(lambda i: fd_length(i))

df['tld'] = df['urls'].apply(lambda i: get_tld(i,fail_silently=True))
def tld_length(tld):
    try:
        return len(tld)
    except:
        return 0

df['tld_length'] = df['tld'].apply(lambda i: tld_length(i))
df['tld_length2'] = df['tld'].apply(lambda i: tld_length(i))


del df['tld']

def has_server_in_string(url):
    if 'server' in url.lower():
        return 1
    else:
        return 0
df['has_server'] = df['urls'].apply(lambda i: has_server_in_string(i))

def has_login_in_string(url):
    if 'login' in url.lower():
        return 1
    else:
        return 0
df['has_login'] = df['urls'].apply(lambda i: has_login_in_string(i))

def has_client_in_string(url):
    if 'client' in url.lower():
        return 1
    else:
        return 0
df['has_client'] = df['urls'].apply(lambda i: has_client_in_string(i))

def has_admin_in_string(url):
    if 'admin' in url.lower():
        return 1
    else:
        return 0
df['has_admin'] = df['urls'].apply(lambda i: has_admin_in_string(i))

def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')
df['count_dir'] = df['urls'].apply(lambda i: no_of_dir(i))
df['count_dir2'] = df['urls'].apply(lambda i: no_of_dir(i))

In [13]:
def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 0
    else:
        return 1
df['short_url'] = df['urls'].apply(lambda i: shortening_service(i))
df['short_url2'] = df['urls'].apply(lambda i: shortening_service(i))
df['short_url3'] = df['urls'].apply(lambda i: shortening_service(i))


def number_subdomain(url):
    count = 0
    domain = urlparse(url).netloc
    dom = '.'.join(domain.split('.')[:-2])
    for i in dom:
        if i=='.':
            count = count+1
    return (count-1)

df['number_subdom'] = df['urls'].apply(lambda i: number_subdomain(i))

# importing libraries for domain features

In [14]:
from urllib.parse import urlparse
from math import log
from string import ascii_lowercase

# domain features (lexical)

In [15]:
def domain_len(url):
    domain = urlparse(str(url)).netloc
    return len(domain)
df['domain_length']= df['urls'].apply(lambda i: domain_len(i))

def dom(url):
    return (urlparse(str(url)).netloc)
df['domain_name'] = df['urls'].apply(lambda i: dom(i))
df['dots_in_domain'] = df['domain_name'].apply(lambda i: i.count('.'))
df['hyphens_in_domain'] = df['domain_name'].apply(lambda i: i.count('-'))

del df['domain_name']


In [16]:
df['url_type'] = df['type'].apply(lambda i: i)


In [17]:
del df['type']


# Saving the dataset as csv file

In [18]:
df.to_csv(r'D:/SASTRA_UNIVERSITY/Mini_Project/testing1.csv')
df_clean = df.loc[:,['ip_in_URL','random_words','avg_len_random_words','english_words','avg_len_english_words','http_in_URL','semicolon_in_URL','AND_in_URL','URL_length','alphabets_in_URL','uppercase_letters_inURL','lowercase_letters_inURL','uppercase_letters_ratio_inURL','lowercase_letters_ratio_inURL','Special_char_in_URL','numbers_ratio_inURL','alphabet_ratio_inURL','fd_length','fd_length2','tld_length','tld_length2','has_server','has_login','has_client','has_admin','count_dir','count_dir2','domain_length','dots_in_domain','short_url','short_url2','short_url3','number_subdom','hyphens_in_domain','url_type']]
df_clean.to_csv(r'D:/SASTRA UNIVERSITY/Mini_Project/testing2.csv')

# SelectK best algorithm for feature reduction

In [20]:
from sklearn.feature_selection import SelectKBest, chi2
data = pd.read_csv(r'D:/SASTRA_UNIVERSITY/Mini_Project/testing1.csv')
X = data.iloc[:, 2:-1]
y = data.iloc[:, -1]


selector = SelectKBest(chi2,k=20)


selected_cols = X.columns[selector.get_support()]

print(selected_cols)

NotFittedError: This SelectKBest instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

# Correlation matrix

In [ ]:
cols_of_interest = ['underscore','//','@','https','hash','numbers_inURL','Special_char_ratio']
corr_matrix = df.corr()
co_mat = corr_matrix.loc[:,:]
co_mat_clean = corr_matrix.loc[:,cols_of_interest]

co_mat.to_csv('D:\SASTRA UNIVERSITY\Mini_Project\correlation_unclean.csv')
co_mat_clean.to_csv('D:\SASTRA UNIVERSITY\Mini_Project\correlation_clean.csv')

# Plotting heat maps

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(co_mat, cmap='YlGnBu')

plt.title('27x27')
plt.xlabel('X-axis label')
plt.ylabel('Y-axis label')

plt.show()

sns.heatmap(co_mat_clean, cmap='YlGnBu')

plt.title('27x7')
plt.xlabel('X-axis label')
plt.ylabel('Y-axis label')

plt.show()